# Tekstklassificeringsopgave

I dette modul starter vi med en simpel tekstklassificeringsopgave baseret på **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**-datasættet: vi vil klassificere nyhedsoverskrifter i en af 4 kategorier: Verden, Sport, Erhverv og Videnskab/Teknologi.

## Datasættet

For at indlæse datasættet vil vi bruge **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API'en.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Vi kan nu få adgang til trænings- og testdelene af datasættet ved at bruge `dataset['train']` og `dataset['test']` henholdsvis:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Lad os udskrive de første 10 nye overskrifter fra vores datasæt:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Tekstvektorisering

Nu skal vi konvertere tekst til **tal**, der kan repræsenteres som tensorer. Hvis vi ønsker repræsentation på ordniveau, skal vi gøre to ting:

* Brug en **tokenizer** til at opdele tekst i **tokens**.
* Byg et **ordforråd** af disse tokens.

### Begrænsning af ordforrådets størrelse

I eksemplet med AG News-datasættet er ordforrådets størrelse ret stor, mere end 100.000 ord. Generelt set har vi ikke brug for ord, der sjældent forekommer i teksten — kun få sætninger vil indeholde dem, og modellen vil ikke lære af dem. Derfor giver det mening at begrænse ordforrådets størrelse til et mindre antal ved at angive et argument til vektoriseringskonstruktøren:

Begge disse trin kan håndteres ved hjælp af **TextVectorization**-laget. Lad os instantiere vektoriseringsobjektet og derefter kalde `adapt`-metoden for at gennemgå al tekst og opbygge et ordforråd:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Bemærk** at vi kun bruger et udsnit af hele datasættet til at opbygge et ordforråd. Vi gør dette for at fremskynde udførelsestiden og undgå at holde dig ventende. Dog tager vi risikoen for, at nogle af ordene fra det samlede datasæt ikke bliver inkluderet i ordforrådet og vil blive ignoreret under træningen. Derfor bør brug af hele ordforrådets størrelse og gennemgang af hele datasættet under `adapt` øge den endelige nøjagtighed, men ikke væsentligt.

Nu kan vi få adgang til det faktiske ordforråd:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Ved hjælp af vektorisatoren kan vi nemt kode enhver tekst til et sæt tal:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Bag-of-words tekstrepræsentation

Fordi ord repræsenterer betydning, kan vi nogle gange forstå meningen med en tekst ved blot at se på de enkelte ord, uanset deres rækkefølge i sætningen. For eksempel, når man klassificerer nyheder, er ord som *vejr* og *sne* sandsynligvis indikatorer for *vejrudsigter*, mens ord som *aktier* og *dollar* peger på *finansielle nyheder*.

**Bag-of-words** (BoW) vektorrepræsentation er den mest simple og letforståelige traditionelle vektorrepræsentation. Hvert ord er knyttet til en vektorindeks, og et vektorelement indeholder antallet af forekomster af hvert ord i et givent dokument.

![Billede, der viser, hvordan en bag-of-words vektorrepræsentation er repræsenteret i hukommelsen.](../../../../../translated_images/da/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Du kan også tænke på BoW som en sum af alle one-hot-enkodede vektorer for de enkelte ord i teksten.

Nedenfor er et eksempel på, hvordan man genererer en bag-of-words repræsentation ved hjælp af Scikit Learn python-biblioteket:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Vi kan også bruge Keras-vectorizeren, som vi definerede ovenfor, til at konvertere hvert ordnummer til en one-hot encoding og lægge alle disse vektorer sammen:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Bemærk**: Du kan blive overrasket over, at resultatet adskiller sig fra det tidligere eksempel. Årsagen er, at i Keras-eksemplet svarer længden af vektoren til størrelsen på ordforrådet, som blev opbygget ud fra hele AG News-datasættet, mens vi i Scikit Learn-eksemplet opbyggede ordforrådet direkte ud fra prøve-teksten.


## Træning af BoW-klassifikatoren

Nu hvor vi har lært, hvordan man opbygger bag-of-words-repræsentationen af vores tekst, lad os træne en klassifikator, der bruger den. Først skal vi konvertere vores datasæt til en bag-of-words-repræsentation. Dette kan opnås ved at bruge `map`-funktionen på følgende måde:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Lad os nu definere et simpelt klassifikations-neuralt netværk, der indeholder ét lineært lag. Inputstørrelsen er `vocab_size`, og outputstørrelsen svarer til antallet af klasser (4). Fordi vi løser en klassifikationsopgave, er den endelige aktiveringsfunktion **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Da vi har 4 klasser, er en nøjagtighed på over 80% et godt resultat.

## Træning af en klassifikator som ét netværk

Fordi vectorizeren også er et Keras-lag, kan vi definere et netværk, der inkluderer det, og træne det fra ende til anden. På denne måde behøver vi ikke at vectorisere datasættet ved hjælp af `map`, vi kan blot give det originale datasæt som input til netværket.

> **Note**: Vi skal stadig anvende maps på vores datasæt for at konvertere felter fra ordbøger (såsom `title`, `description` og `label`) til tuples. Men når vi indlæser data fra disk, kan vi fra starten opbygge et datasæt med den nødvendige struktur.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrammer, trigrammer og n-grammer

En begrænsning ved bag-of-words-metoden er, at nogle ord indgår i flertalsudtryk, for eksempel har ordet 'hot dog' en helt anden betydning end ordene 'hot' og 'dog' i andre sammenhænge. Hvis vi altid repræsenterer ordene 'hot' og 'dog' med de samme vektorer, kan det forvirre vores model.

For at løse dette bruges **n-gram repræsentationer** ofte i metoder til dokumentklassifikation, hvor frekvensen af hvert ord, to-ords eller tre-ords udtryk er en nyttig funktion til at træne klassifikatorer. I bigram-repræsentationer, for eksempel, tilføjer vi alle ordpar til ordforrådet, ud over de oprindelige ord.

Nedenfor er et eksempel på, hvordan man genererer en bigram bag-of-words repræsentation ved hjælp af Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Den største ulempe ved n-gram-metoden er, at ordforrådets størrelse begynder at vokse ekstremt hurtigt. I praksis er vi nødt til at kombinere n-gram-repræsentationen med en dimensionalitetsreduktionsteknik, såsom *embeddings*, som vi vil diskutere i næste enhed.

For at bruge en n-gram-repræsentation i vores **AG News**-datasæt, skal vi angive `ngrams`-parameteren til vores `TextVectorization`-konstruktør. Længden af et bigram-ordforråd er **betydeligt større**, i vores tilfælde er det mere end 1,3 millioner tokens! Derfor giver det mening også at begrænse bigram-tokens til et rimeligt antal.

Vi kunne bruge den samme kode som ovenfor til at træne klassifikatoren, men det ville være meget hukommelsesineffektivt. I næste enhed vil vi træne bigram-klassifikatoren ved hjælp af embeddings. I mellemtiden kan du eksperimentere med træning af bigram-klassifikatoren i denne notebook og se, om du kan opnå højere nøjagtighed.


## Automatisk beregning af BoW-vektorer

I eksemplet ovenfor beregnede vi BoW-vektorer manuelt ved at summere one-hot-enkodningerne af individuelle ord. Den nyeste version af TensorFlow giver os dog mulighed for automatisk at beregne BoW-vektorer ved at angive parameteren `output_mode='count` til vektoriseringens konstruktør. Dette gør det betydeligt lettere at definere og træne vores model:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Termfrekvens - invers dokumentfrekvens (TF-IDF)

I BoW-repræsentationen vægtes ordforekomster ved hjælp af den samme teknik, uanset hvilket ord der er tale om. Det er dog tydeligt, at hyppige ord som *en* og *i* er langt mindre vigtige for klassifikation end specialiserede termer. I de fleste NLP-opgaver er nogle ord mere relevante end andre.

**TF-IDF** står for **termfrekvens - invers dokumentfrekvens**. Det er en variation af bag-of-words, hvor man i stedet for en binær 0/1-værdi, der angiver, om et ord optræder i et dokument, bruger en flydende værdi, som relaterer sig til hyppigheden af ordets forekomst i korpuset.

Mere formelt defineres vægten $w_{ij}$ af et ord $i$ i dokumentet $j$ som:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
hvor
* $tf_{ij}$ er antallet af forekomster af $i$ i $j$, dvs. den BoW-værdi, vi har set tidligere
* $N$ er antallet af dokumenter i samlingen
* $df_i$ er antallet af dokumenter, der indeholder ordet $i$ i hele samlingen

TF-IDF-værdien $w_{ij}$ stiger proportionalt med, hvor mange gange et ord optræder i et dokument, og justeres i forhold til antallet af dokumenter i korpuset, der indeholder ordet. Dette hjælper med at tage højde for, at nogle ord optræder hyppigere end andre. For eksempel, hvis ordet optræder i *alle* dokumenter i samlingen, er $df_i=N$, og $w_{ij}=0$, og disse termer vil blive fuldstændigt ignoreret.

Du kan nemt oprette TF-IDF-vektorisering af tekst ved hjælp af Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

I Keras kan `TextVectorization`-laget automatisk beregne TF-IDF-frekvenser ved at angive parameteren `output_mode='tf-idf'`. Lad os gentage koden, vi brugte ovenfor, for at se, om brug af TF-IDF øger nøjagtigheden:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Konklusion

Selvom TF-IDF-repræsentationer giver vægt til forskellige ord baseret på deres hyppighed, er de ikke i stand til at repræsentere betydning eller rækkefølge. Som den berømte lingvist J. R. Firth sagde i 1935: "Den fulde betydning af et ord er altid kontekstuel, og ingen undersøgelse af betydning uden for konteksten kan tages seriøst." Senere i kurset vil vi lære, hvordan man fanger kontekstuel information fra tekst ved hjælp af sprogmodellering.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at sikre nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for eventuelle misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
